# Preprocessing des images .TIF du dataset lnria Aerial Image Labeling Dataset
# Roof Detection / Segmentation

5000x5000 px en résolution 0.3 m /px vers 1024x1024 px en résolution 0.6 m /px

21/11/21
convertir toutes les images du dataset INRIA en 1024 px 0.6 m/px et annotations associées au format VGG Annotation Tool en json 

Création du fichier via_region_data.json pour stocker toutes les annotations de toutes les images considérées dans l'étude

In [8]:
import os 
from osgeo import gdal 
import json
import copy
import base64
from PIL import Image
from tqdm import tqdm 
import cv2
import numpy as np
import sys
import imutils

In [9]:
dataset_dir = r"C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset"

train_dir = os.path.join(dataset_dir, 'train', 'images')
gt_dir    = os.path.join(dataset_dir, 'train', 'gt')
test_dir  = os.path.join(dataset_dir, 'test', 'images')

In [ ]:
filepath = os.path.join(dataset_dir, 'train', 'gt', 'austin16.tif')
!gdalinfo $filepath

## Convert GeoTIFF to 0.6 m resolution GeoTIFF

In [7]:
def res(dataset_dir, res_dir, file_name, res=0.6):
    file_path = os.path.join(dataset_dir, file_name)
    name, ext = file_name.split('.')
    res_path = os.path.join(res_dir, name+'_s.'+ext)
    !gdalwarp -tr $res $res $file_path $res_path

In [ ]:
tmp_path = os.path.join(dataset_dir, 'tmpINRIA')
if not os.path.isdir(tmp_path):
    os.mkdir(tmp_path)
    os.mkdir(os.path.join(tmp_path, 'images'))
    os.mkdir(os.path.join(tmp_path, 'gt'))

lst = os.listdir(train_dir)
for i in lst:
    if i.startswith('austin') or i.startswith('chicago') or i.startswith('vienna') or i.startswith('tyrol'):
        res(train_dir, os.path.join(tmp_path, 'images'), i, res=0.6)
        res(gt_dir, os.path.join(tmp_path, 'gt'), i, res=0.6)

## Convert GeoTIFF to 1024 px side GeoTIFF

In [16]:
def GetExtent(ds):
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel
    return round(xmin,0), round(xmax,0), round(ymin,0), round(ymax, 0)
# https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings

In [13]:
def crop(dataset_dir, final_dir, file_name):
    file_path = os.path.join(dataset_dir, 'images', file_name)
    gt_path = os.path.join(dataset_dir, 'gt', file_name)
    
    name, exte = file_name.split('.')
    raster = gdal.Open(file_path)
    # print(file_path)
    ext = GetExtent(raster)
    Xdim, Ydim = raster.RasterXSize, raster.RasterYSize
    for i in range(Xdim//1024+1):
        for j in range(Ydim//1024+1):
            final_name = name[:-2]+ '_'+str(i)+str(j)+'.'+exte
            final_path = os.path.join(final_dir, 'images', final_name)
            final_gt_path = os.path.join(final_dir, 'gt', final_name)
            
            if i==Xdim//1024 or j==Ydim//1024:
                if i==Xdim//1024 and j!=Ydim//1024:
                    nxmin = ext[1] - 1024 * 0.6
                    nxmax = ext[1]
                    nymin = ext[3] - 1024 * 0.6 * (j+1)
                    nymax = ext[3] - 1024 * 0.6 * j
                    #pxlim = [Xdim - 1024, Xdim, 0, 1024]
                    
                elif i!=Xdim//1024 and j==Ydim//1024:
                    nxmin = ext[0] + 1024 * 0.6 * i
                    nxmax = ext[0] + 1024 * 0.6 * (i+1)
                    nymin = ext[2]
                    nymax = ext[2] + 1024 * 0.6
                    #pxlim = [0, 1024, Ydim - 1024, Ydim]
                    
                elif i==Xdim//1024 and j==Ydim//1024:
                    nxmin = ext[1] - 1024 * 0.6
                    nxmax = ext[1]
                    nymin = ext[2]
                    nymax = ext[2] + 1024 * 0.6
                    #pxlim = [Xdim - 1024, Xdim, Ydim - 1024, Ydim]
                
            else:
                nxmin = ext[0] + 1024 * 0.6 * i
                nxmax = ext[0] + 1024 * 0.6 * (i+1)
                nymin = ext[3] - 1024 * 0.6 * (j+1)
                nymax = ext[3] - 1024 * 0.6 * j
                #pxlim = [1024*i, 1024*(i+1), 1024*j, 1024*(j+1)]
                
            !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $file_path $final_path
            !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $gt_path $final_gt_path
            
            #json_path = os.path.join(dataset_dir, name + '.json')
            #final_json_path = os.path.join(final_dir, name[:-2]+ '_'+str(i)+str(j)+'.json')
            #CropAnnot(json_path, final_json_path, final_path, final_name, pxlim)

In [12]:
print(dataset_dir)

C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset


In [13]:
tmp_path = os.path.join(dataset_dir, 'tmpINRIA')
print('tmp path is {}'.format(tmp_path))
preprocess_path = os.path.join(dataset_dir, 'preprocessINRIA')
if not os.path.isdir(preprocess_path):
    os.mkdir(preprocess_path)
    os.mkdir(os.path.join(preprocess_path, 'images'))
    os.mkdir(os.path.join(preprocess_path, 'gt'))

tmp path is C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\tmpINRIA


In [ ]:
lst = os.listdir(os.path.join(tmp_path, 'images'))
for geo in lst:
    crop(tmp_path, preprocess_path, geo)

In [ ]:
filepath = os.path.join(preprocess_path, 'images', 'vienna15_02.tif')
!gdalinfo $filepath

## Transform Annotations from TIF binary tiles to VGG annotation tool json file

In [5]:
print(os.getcwd())

C:\Users\VArri\Documents\GitHub\RoofTop\samples\darkroof


In [15]:
print(preprocess_path)

C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\preprocessINRIA


In [6]:
import random 
def Annot(preprocess_path, res=0.6, surf=100, eps=0.01):
    
    john=True
    stats=[]
    jsonf = {} # only one big annotation file
    
    with open(os.path.join(preprocess_path,'via_region_data.json'), 'w') as js_file:
        gt_path = os.path.join(preprocess_path, 'gt')
        images_path = os.path.join(preprocess_path, 'images')
        
        # All the elements in the images folders
        lst = os.listdir(images_path)
        for elt in tqdm(lst, desc='lst'):
        
            # Read the binary mask, and find the contours associated
            gray = cv2.imread(os.path.join(gt_path, elt))
            imgray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(imgray, 127, 255, 0)
            contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            # Only keep contours with a minimum surface to avoid overload in preprocessing
            # Also mandatory to reduce the length of contours
            # https://www.pyimagesearch.com/2021/10/06/opencv-contour-approximation/
            # Contours approximation based on Ramer–Douglas–Peucker (RDP) algorithm
            areas = [cv2.contourArea(contours[idx])*res*res for idx in range(len(contours))]
            large_contour = []
            for i in range(len(areas)):
                if areas[i]>surf:
                    large_contour.append(contours[i])
            # large_contour = [contours[i] if areas[i]>surf else contours[0] for i in range(len(areas))]
            approx_contour = [cv2.approxPolyDP(c, eps * cv2.arcLength(c, True), True) for c in large_contour]
            # print('Taille des contours larges {}'.format([len(large_contour[i]) for i in range(len(large_contour))]))
            # print('Taille des contours approx {}'.format([len(approx_contour[i]) for i in range(len(approx_contour))]))
            # Stats on ApproxPolyDP to approximate teh contour, in particular the number of points
            for idx in range(len(large_contour)):
                stats.append((len(large_contour[idx])-len(approx_contour[idx]))/len(large_contour[idx]))
            
            if False: # random.randint(0,5)==3:

                im = cv2.imread(os.path.join(images_path, elt))
                cv2.drawContours(im, large_contour, -1, (0,255,0), 3)
                # cv2.drawContours(im, approx_contour, -1, (0,255,0), 3)
                cv2.imshow('contours', im)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            
            #print(approx_contour)
            #print(approx_contour[0])
            #print(approx_contour[0][:,0])
            #print(approx_contour[0][:,0][:,1])
            
            # -------------------------------------------------------------------------------
            # BUILDING VGG ANNTOTATION TOOL ANNOTATIONS LIKE 
            if len(approx_contour) > 0:
                
                                    
                regions = [0 for i in range(len(approx_contour))]
                for i in range(len(approx_contour)):
                    # print('ok')
                    # print(i)
                    shape_attributes = {}
                    region_attributes = {}
                    regionsi = {}
                    shape_attributes['name'] = 'polygon'
                    shape_attributes['all_points_x'] = approx_contour[i][:, 0][:, 0].tolist()
                    # https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable
                    shape_attributes['all_points_y'] = approx_contour[i][:, 0][:, 1].tolist()
                    # print(json.dumps(shape_attributes, indent=4))
                    regionsi['shape_attributes'] = shape_attributes
                    regionsi['region_attributes'] = region_attributes
                    regions[i] = regionsi

                size = os.path.getsize(os.path.join(images_path, elt))
                name = elt + str(size)
                json_elt = {}
                json_elt['filename'] = elt
                json_elt['size'] = str(size)
                json_elt['regions'] = regions
                json_elt['file_attributes'] = {}
                # print(json.dumps(json_elt, indent=4))

                jsonf[name] = json_elt
        
                
            if len(approx_contour) ==3 and john:
                print(json.dumps(jsonf, indent=4))
                john = False
                
        json.dump(jsonf, js_file) ### EXPLAIN A LITTLE BIT MORE
    
    # Stats processing 
    print('ApproxPolyDP is decreasing contours number of points by {} % on average'.format(np.around(np.mean(stats)*100),1))

In [17]:
Annot(os.path.join(preprocess_path, 'colab', 'val'))

lst:  71%|████████████████████████████████████████████████████▉                      | 185/262 [00:06<00:05, 13.51it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

lst: 100%|███████████████████████████████████████████████████████████████████████████| 262/262 [00:08<00:00, 30.59it/s]


ApproxPolyDP is decreasing contours number of points by 69.0 % on average


In [112]:
print(json.dumps(data['0030fd0e6378.png236322']['regions'][0]['shape_attributes']['all_points_x'][0], indent=4))

291


In [20]:
jsonpath = r"C:\Users\VArri\Downloads"
with open(os.path.join(jsonpath, 'via_export_json.json'), 'r') as op:
    data = json.load(op)
    print(json.dumps(data, indent=4))
    #for i in data['0030fd0e6378.png236322']:
        # print(i)
        # pour lire les infos directement dans les paniers 

{
    "AEC255F4-7883-4EE9-9DDB-F8216AD6613F.png9259408": {
        "filename": "AEC255F4-7883-4EE9-9DDB-F8216AD6613F.png",
        "size": 9259408,
        "regions": [
            {
                "shape_attributes": {
                    "name": "polygon",
                    "all_points_x": [
                        291,
                        531,
                        536,
                        298,
                        230
                    ],
                    "all_points_y": [
                        696,
                        686,
                        787,
                        797,
                        898
                    ]
                },
                "region_attributes": {}
            }
        ],
        "file_attributes": {}
    },
    "0030fd0e6378.png236322": {
        "filename": "0030fd0e6378.png",
        "size": 236322,
        "regions": [
            {
                "shape_attributes": {
                    "name": "polygon",
  

## Shuffling and split annotated files in train, val, test folders 

In [18]:
dataset_dir

'C:\\Users\\VArri\\Documents\\Rooftop\\dataset\\dataset\\dataset'

In [11]:
from random import shuffle
from random import randint
from tqdm import tqdm

colab_path = os.path.join(dataset_dir, 'preprocessINRIA', 'colab')
if not os.path.isdir(colab_path):
    os.mkdir(colab_path)
    os.mkdir(os.path.join(colab_path, 'train'))
    os.mkdir(os.path.join(colab_path, 'train', 'images'))
    os.mkdir(os.path.join(colab_path, 'train', 'gt'))
    os.mkdir(os.path.join(colab_path, 'val'))
    os.mkdir(os.path.join(colab_path, 'val', 'images'))
    os.mkdir(os.path.join(colab_path, 'val', 'gt'))
    # os.mkdir(os.path.join(colab_path, 'test'))
    #os.mkdir(os.path.join(colab_path, 'test', 'images'))
    #os.mkdir(os.path.join(colab_path, 'test', 'gt'))
                          
    
from shutil import copyfile
img_path = os.path.join(dataset_dir, 'preprocessINRIA', 'images')
gt_path = os.path.join(dataset_dir, 'preprocessINRIA', 'gt')
lst = os.listdir(img_path)
shuffle(lst)
for geo in tqdm(lst, desc='lst'):
    if geo.endswith('.tif'):
        name, exte = geo.split('.')
        if True:#os.path.isfile(os.path.join(preprocess_path, name+'.json')):
            var = randint(0, 9)
            if var in range(0,8):
                folder = 'train'                
            elif var in range(8,10):
                folder = 'val'            
            # elif var==0:
            #     folder = 'test'
                
            copyfile(os.path.join(img_path,geo), os.path.join(colab_path,folder,'images', geo))
            copyfile(os.path.join(gt_path,geo), os.path.join(colab_path,folder, 'gt', geo))

lst: 100%|█████████████████████████████████████████████████████████████████████████| 1296/1296 [01:19<00:00, 16.34it/s]
